In [2]:
import xarray as xr
import numpy as np
from PIL import Image

In [3]:


# Path to your downloaded ETOPO file (NetCDF or GMT .grd). Adjust as needed:
ETOPO_FILE = "ETOPO1_Ice_g_gmt4.grd"  # or .nc, etc.

# Output filename for the PNG:
OUTPUT_PNG = "earth_heightmap_0to255.png"

# -----------------------------------------------------------
# 1) Load the ETOPO data
ds = xr.open_dataset(ETOPO_FILE)

# ETOPO can store the elevation data under different variable names,
# e.g. 'z' or 'Band1' or 'elevation'.
# Inspect ds.data_vars or print(ds) to find the correct var name.
# For ETOPO1 "Ice_g" .grd file, it's often called 'z'.
var_name = 'z'
if var_name not in ds.variables:
    # If it's not 'z', pick the correct one. Or just do ds.data_vars
    print("Available data vars:", ds.data_vars)
    raise ValueError("Adjust var_name to match your file.")

data = ds[var_name].values  # 2D numpy array: (lat x lon or vice versa)

# -----------------------------------------------------------
# 2) Find global min/max to map
# Deepest ocean ~ -11034 m, top of Everest ~ +8849 m,
# but let's rely on the actual data range in this file:
min_val = np.nanmin(data)  # typically around -11034 or so
max_val = np.nanmax(data)  # typically around +8849 or so
print(f"Data range: {min_val} to {max_val}")

# -----------------------------------------------------------
# 3) Normalize the data to [0..1], then scale to [0..255]
# We'll clamp or just trust the dataset range.
data_clamped = np.clip(data, min_val, max_val)

norm_data = (data_clamped - min_val) / (max_val - min_val)  # in [0..1]
gray_data = (norm_data * 255.0).astype(np.uint8)

# -----------------------------------------------------------
# 4) Convert to image and save
# The data might be huge (e.g., 10800x5400 for 1 arc-minute),
# be mindful of memory. Pillow can handle it if there's enough RAM.
img = Image.fromarray(gray_data, mode='L')
img.save(OUTPUT_PNG)

print(f"Saved heightmap to {OUTPUT_PNG}")
ds.close()


ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html